# TP2: A deeper understanding CNN denoising


The objective of this lesson is to study more in detail some of the top CNN image denoising algorithms. 

We will cover the following topics:
* Study some aspects of the DnCNN network
* Test FFDNet
* The role of the training loss
* The noise-to-noise training strategy

There are **6 questions** in the notebook and corresponding text areas to fill-in the answers. 


**Note that** some of the bloks of this notebook take up to three minutes to run on CPU, so be patient.


#### Instructions
To solve this TP, answer the questions below. Then export the notebook with the answers using  the menu option **File->Download as->HTML**. Send the resulting *html* file by mail to [facciolo@cmla.ens-cachan.fr](mailto:facciolo@cmla.ens-cachan.fr) with subject "Report tp1 of SURNAME, Name", by 23/11/2018.  You will receive an acknowledgement of receipt.

In [ ]:
# Setup code for the notebook

# Execute code 'cells' like this by clicking on the 'Run' 
# button or by pressing [shift] + [Enter].

# This cell only imports some python packages that will be
# used below. It doesn't generate any output. Something similar 
# applies to the next two or three cells. They only define 
# functions that are used later.


# This notebook can also run on colab (https://colab.research.google.com/)
# The following lines install the necessary packages in the colab environment
try:
    from google.colab import files
    !pip install torch==0.4.1
    !pip install torchvision
    !pip install Pillow==4.0.0
    !pip install scikit-image
    !pip install hdf5storage

    !rm -fr MVAdenoising2018
    !git clone  https://github.com/gfacciol/MVAdenoising2018
    !cp -r MVAdenoising2018/* .

except ImportError:
    # %matplotlib notebook
    pass


# These are all the includes used through the notebook
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import vistools          # image visualization toolbox
from   skimage import io # read and write images

# global variable for setting the torch.load    map_location
if torch.cuda.is_available():
    loadmap = {'cuda:0': 'gpu'}
else:
    loadmap = {'cuda:0': 'cpu'}
    
#%matplotlib notebook
# Autoreload external python modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

-----------------------------

# DnCNN

We start by testing some aspects of the DnCNN architecture proposed in:

*K. Zhang, W. Zuo, Y. Chen, D. Meng, and L. Zhang, “Beyond a Gaussian Denoiser: Residual Learning of Deep CNN for Image Denoising,” IEEE Trans. Image Process., vol. 26, no. 7, pp. 3142–3155, Jul. 2017.*

The network has several hidden layers, all of them equal: convolution, **batch normalization** and **ReLU activations** and uses **residual learning**. All convolutions have the same size (the authors used 3x3).

<img width=700 src="https://www.researchgate.net/profile/Yunjin_Chen/publication/306187437/figure/fig4/AS:667093628379148@1536058923422/The-architecture-of-the-proposed-DnCNN-network.png"/>


The module `models` declares the DnCNN network. An instance of the network for grayscale images is created with `model = DnCNN(1,1)` where the parameters `1` indicate the number of input and output channels.
The model is made of atomic blocks such as `nn.Conv2d`, whch represents a convolutional layer. Note how the layers are declared in the `__init__` method of each model and then called in `forward`. 


The implementation of DnCNN can be found in the `models/DnCNN.py` file. The model constructor can be called as follows:

```python
class DnCNN(nn.Module):
    '''PyTorch module for the DnCNN network.'''
    def __init__(self, in_channels=1, out_channels=1, num_layers=17, 
                 features=64, kernel_size=3, residual=True):
        '''
        Constructor for a DnCNN network.

        Args:
            - in_channels: input image channels (default 1)
            - out_channels: output image channels (default 1)
            - num_layers: number of layers (default 17)
            - num_features: number of hidden features (default 64)
            - kernel_size: size of conv. kernel (default 3)
            - residual: use residual learning (default True)

        Return: network with randomly initialized weights
        '''
```

A newly created network is initialized with random weights.
We also provide you with a function to load pretrained weights

```python
def DnCNN_pretrained(sigma=30, savefile=None, verbose=False, color=False):
    '''
    Loads the pretrained weights of DnCNN for grayscale and color images  
    from https://github.com/cszn/DnCNN.git

    Args:
        - sigma   : is the level of noise in range(10,76,5)
        - savefile: is the .pt file to save the model weights 
        - verbose : verbose output
        - color   : load the weights for the color networks

    Returns:
        - DnCNN(1,1) model with 17 layers with the pretrained weights    
        or
        - DnCNN(3,3) model with 20 layers with the pretrained weights     
    '''
```

### Recap: Extra credit from TP1 - training of a small DnCNN

As an optional assignment, you were asked to train a small DnCNN network with the same receptive field and roughly the same number of parameters than a DCT denoiser with 8x8 patches.

In case you didn't do the assignment, you can now load a pre-trained network and compare its results with a pretrained Full DnCNN.

In [ ]:
# compare the evolution of the loss
from models import DnCNN, CONV_BN_RELU

# load last checkpoint
sigma   = 30
net = torch.load('pre-trained-tp2/tiny_DnCNN_2000.pt', map_location=loadmap)

plt.semilogy(net[3], '.-', label='net1 val')
plt.semilogy(net[2], '.-', label='net1 train')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('loss');

Let's see the evolution of the results along the epochs for this tiny DnCNN, compare the result with the one of our best DCT denoising network, and with the DnCNN network trained by the authors.   

In [ ]:
# evolution of the denoising performance during training
from denoising_helpers import test_denoiser, PSNR
from models import DCTlike, DnCNN_pretrained
from skimage import io
from vistools import unzip

# load an image and compute noisy one 
sigm = 30 
img_clean = io.imread('datasets/BSD68/test002.png', dtype='float32')
img_noisy = img_clean + np.random.normal(0, sigma, img_clean.shape)

# outputs list of pairs (image, text)
outs   = list()

# add noisy and clean images
outs.append( (img_noisy, 'noisy') )
outs.append( (img_clean, 'clean') )

# add results of iterations
for i in range(0,2001,500):
    print('%d '%i, end='')
    net = torch.load('pre-trained-tp2/tiny_DnCNN_%04d.pt' % i, map_location=loadmap)[0]
    out = test_denoiser(net, img_noisy, sigma, has_noise=True)[0]
    outs.append( (out, 'trained tiny DnCNN - it %d - %f (dB)' % (i, PSNR(out, img_clean)) ) ) 


# add result of original dct
net = DCTlike(8, sigma, initializeDCT=True)
out = test_denoiser(net, img_noisy, sigma, has_noise=True)[0]
outs.append( (out, 'original dct - %f (dB)' % (PSNR(out, img_clean)) ) )

# add result of full DnCNN with trained weights
net = DnCNN_pretrained(sigma)
out = test_denoiser(net, img_noisy, sigma, has_noise=True)[0]
outs.append( (out, 'Full DnCNN - %f (dB)' % (PSNR(out, img_clean)) ) )


# show result as a gallery 
vistools.display_gallery(unzip(outs,0), unzip(outs,1))

# Stress tests on DnCNN: robustness to changes in $\sigma$

We will consider the DnCNN network trained for $\sigma = \sigma_0$ and see how it responds to changing $\sigma$. The following code applies noise of different standard deviation to an image and tests two networks: one trained for $\sigma_0$ and the other one using the network trained for the closest $\sigma$. The networks have been pretrained for $\sigma = 10, 15, ..., 70, 75$.

In [ ]:
from models import DnCNN, DnCNN_pretrained
from denoising_helpers import test_denoiser, PSNR
from vistools import unzip

# load an image (change the number to test other images)
image = io.imread('datasets/BSD68/test002.png', dtype='float32')

# set a noise level
sigma0 = 30

im_clean = image.astype('float32')
noise = np.random.normal(0, 1., im_clean.shape)

# load pre-trained DnCNN
dncnn_sigma0 = DnCNN_pretrained(sigma0)

sigmas = np.linspace(10,75,14)   #sigmas = np.linspace(5,80,76)
#print(sigmas)

PSNR_dncnn_sigma0 = list()
PSNR_dncnn_sigma  = list()
# outputs list of pairs (image, text)
outputs = list()

# apply them to the image
for i, sigma in enumerate(sigmas):
    print('%d '%i, end='')
    
    # add noise
    im_noisy = im_clean + sigma * noise

    # load correct model
    dncnn_sigma = DnCNN_pretrained(sigma)
    
    # denoise image with both models
    out_sigma0 = test_denoiser(dncnn_sigma0, im_noisy, None, has_noise=True)[0]
    out_sigma  = test_denoiser(dncnn_sigma , im_noisy, None, has_noise=True)[0] 
    
    PSNR_dncnn_sigma0.append( PSNR(out_sigma0, im_clean) )
    PSNR_dncnn_sigma.append(  PSNR(out_sigma , im_clean) )
    
    outputs.append( (out_sigma0, 'noise sigma = %f - model sigma = %f - PSNR = %f' %(sigma, sigma0, PSNR_dncnn_sigma0[i])) )    
    outputs.append( (out_sigma,  'noise sigma = %f - model sigma = %f - PSNR = %f' %(sigma, sigma,  PSNR_dncnn_sigma[i])) ) 
    
# show as a gallery
if len(outputs)/2 < 20:
    vistools.display_gallery(unzip(outputs,0), unzip(outputs,1))

plt.plot(sigmas, PSNR_dncnn_sigma0, '.-', label='model with sigma0')
plt.plot(sigmas, PSNR_dncnn_sigma , '.-', label='model with correct sigma')
plt.legend(); plt.xlabel('sigma'); plt.ylabel('PSNR (dB)');

This result is reasonable. Note that the denoising images obtained with a fixed $\sigma_0$ are oversmoothed when $\sigma < \sigma_0$ and increasingly noisy for $\sigma > \sigma_0$. In both cases the PSNR is below the one obtained for the correct $\sigma$. The bumps in the curve with the correct sigma are due to the quantization values of pretrained networks. 

## Stress tests on DnCNN: shifting the input

In this experiment we study the performance of the DnCNN algorithm as the intensity range of the input image $u$ is shifted. 

For that we will take an image $u$ and first reduce its initial intensity range [0,255] by a constant factor of 5 in order to obtain an image with values in the range [0, 51]. This reduced range allows to shift the intensty without saturating. The intensity of this image can then be shifted by a constant $b \in [0,200]$: $$\widetilde u = u/5 + b.$$ 
This range of $b$ assures that $\widetilde u$ is in the range [0,255].  For this experiment we will fix $\sigma_0=10$. 

**Question 1.** <!-- Use the code of the previous block as inspiration for--> The following code creates a plot of the PSNR as a function of $b$ for values of $b \in [0,200]$ (`bs = np.linspace(0,200,12)`). What can you say about the denoiser performance? In a second plot explore more extreme intensity shifts with $b \in [-50, 230]$ (`bs = np.linspace(-50,230,14)`). Can you explain the result?



ANSWERS TO QUESTION 1.

In [ ]:
from models import DnCNN, DnCNN_pretrained
from denoising_helpers import *
from vistools import unzip

# load an image (change the number to test other images)
image = io.imread('datasets/BSD68/test004.png', dtype='float32')

# set a noise level
sigma0 = 10
noise = np.random.normal(0., 1., image.shape)
scale = 5

# load pre-trained DnCNN
dncnn_sigma0 = DnCNN_pretrained(sigma0)

bs = np.linspace(0,200,12)
#bs = np.linspace(-50,230,14)

PSNR_dncnn_sigma_shift = list() 
# outputs list of pairs (image, text)
outputs = list()

# apply them to the image
for i, b in enumerate(bs):
    print('%d '%i, end='')

    # scale clean image and add noise
    im_clean = image/scale + b
    im_noisy = im_clean + sigma0 * noise
     
    # denoise image and compute PSNR
    out_sigma0 = test_denoiser(dncnn_sigma0, im_noisy, None, has_noise=True)[0]
    PSNR_dncnn_sigma_shift.append( PSNR(out_sigma0, im_clean) )
    outputs.append( ( (out_sigma0-b)*scale , 'b = %f - sigma = %f - PSNR = %f' % (b, sigma0, PSNR_dncnn_sigma_shift[i])) )


######
###### NOTE THAT in the gallery the images are scaled back to the range [0,255]   
######

# show as a gallery
if len(outputs) < 80:
    vistools.display_gallery( unzip(outputs,0), unzip(outputs,1) )

plt.plot(bs, PSNR_dncnn_sigma_shift, '.-', label='model with sigma0')
plt.legend(); plt.xlabel('b'); plt.ylabel('PSNR (dB)');

# Stress tests on DnCNN: affine scalings

Now suppose we only have a pre-trained network for a noise level $\sigma_0$. We would like to denoise an image $u$ contaminated with $\sigma$. We can apply an affine scale change to our image so that the standard deviation of the noise becomes $\sigma_0$:
$$u_s = \frac{\sigma_0}{\sigma}(u - \overline{u}) + \overline{u} = \frac{\sigma_0}{\sigma}u + \overline{u}\left(1 - \frac{\sigma_0}{\sigma}\right) = \alpha u + \beta,$$
where $\overline u$ is the average gray level of $u$. The constant $\beta$ was added so that $\overline u_s = \overline u$. The idea is then to apply the denoising network to $u_s$ and then invert the affine transformation:

$$\hat u = \frac1\alpha \mathcal{F}_{\sigma_0}(\alpha u + \beta) - \frac\beta\alpha.$$


In the following experiment we will study how the results of the network vary with $\sigma$.


<br/>
<br/>

**Attention:** For performance reasons, the variables `PSNR_dncnn_sigma0` and `PSNR_dncnn_sigma` of this block are re-used from the block: *"Stress tests on DnCNN: robustness to changes in $\sigma$"*, that was executed before.

In [ ]:
from models import DnCNN, DnCNN_pretrained
from denoising_helpers import *
from vistools import unzip

# load an image (change the number to test other images)
image = io.imread('datasets/BSD68/test002.png', dtype='float32')

# set a noise level
sigma0 = 30

im_clean = image.astype('float32')
noise = np.random.normal(0, 1., im_clean.shape)

# load pre-trained DnCNN
dncnn_sigma0 = DnCNN_pretrained(sigma0)

sigmas = np.linspace(10,75,14)  #sigmas = np.linspace(5,80,76)
#print(sigmas)

##############
### THE PSNR_dncnn_sigma0 and PSNR_dncnn_sigma VARIABLES ARE COMPUTED IN SECTION: 
###    Stress tests on DnCNN: robustness to changes in $\sigma$ 
##############

#PSNR_dncnn_sigma0 = list()
#PSNR_dncnn_sigma  = list()
PSNR_dncnn_scaled = list() 

outputs = list()

# apply them to the image
for i, sigma in enumerate(sigmas):
    print('%d '%i, end='')
    
    # add noise
    im_noisy = im_clean + sigma * noise

    # load correct model
    dncnn_sigma = DnCNN_pretrained(sigma)
    
    # denoise image with both models
    #out_sigma0 = test_denoiser(dncnn_sigma0, im_noisy, None, has_noise=True)[0]
    #out_sigma  = test_denoiser(dncnn_sigma , im_noisy, None, has_noise=True)[0]
    
    # scaling to match model noise level
    a = sigma0/sigma
    b = np.mean(im_noisy)*(1-a)
    out_scaled = test_denoiser(dncnn_sigma0, a*im_noisy + b, None, has_noise=True)[0]
    out_scaled = 1/a*out_scaled - b/a
    
    #PSNR_dncnn_sigma0.append(PSNR(out_sigma0, im_clean))
    #PSNR_dncnn_sigma .append(PSNR(out_sigma , im_clean))
    PSNR_dncnn_scaled.append(PSNR(out_scaled, im_clean))

    #outputs.append( (out_sigma0, 'noise sigma = %f - model sigma = %f - PSNR = %f' % (sigma, sigma0, PSNR_dncnn_sigma0[i])) )
    #outputs.append( (out_sigma,  'noise sigma = %f - model sigma = %f - PSNR = %f' % (sigma, sigma , PSNR_dncnn_sigma [i])) )
    outputs.append( (out_scaled, 'noise sigma = %f - scaled to sigma %f - PSNR = %f' % (sigma, sigma , PSNR_dncnn_scaled[i])) )
    
    
# show as a gallery
if len(outputs) < 80:
    vistools.display_gallery( unzip(outputs,0), unzip(outputs,1) )

plt.plot(sigmas, PSNR_dncnn_sigma0, '.-', label='model with sigma0')
plt.plot(sigmas, PSNR_dncnn_sigma , '.-', label='model with correct sigma')
plt.plot(sigmas, PSNR_dncnn_scaled, '.-', label='model with sigma0, scaled input')
plt.legend(); plt.xlabel('sigma'); plt.ylabel('PSNR (dB)');

Let's compute the $\Delta PSNR$ between the *scaled input* and *correct sigma* outputs.

In [ ]:
deltaPSNR = np.array(PSNR_dncnn_sigma) - np.array(PSNR_dncnn_scaled)

plt.plot(sigmas, deltaPSNR, '.-', label = 'PSNR correct sigma - PSNR scaled')
plt.legend(); plt.xlabel('sigma'); plt.ylabel('Delta PSNR (dB)');

**Question 2.** Can you explain the behavior of the method with scaled input for sigma>=30? What about sigma<30? Do  you see a way to obtain a method with good performance over a wider range of sigma values? 

ANSWER TO QUESTION 2

# The case of DCT denoising

Let's consider the case of the DCT denoising. The DCT is a change of basis in the space of patches. The DCT basis is orthonormal, and let us denote it by $\mathcal B = \{b_1, ..., b_d\}$, where $d$ is the dimension of the patches. We can express the DCT denoising of a patch as:
$$\text{DCT}_{\sigma_0}(P_x u) = \langle P_xu, b_1\rangle b_1 + \sum_{i=2}^d S\left(\langle P_xu,b_i\rangle,\sigma_0\right) b_i,$$
where $S$ is a shrinkage operator of parameter $\sigma_0$. For example, the hard-thresholding operator is given by
$$S_{\text{hard}}(x,\sigma_0) = 
\left\{
\begin{array}{l l}
x & \text{if } x \geq 3\sigma_0, \\
0 & \text{if } x  < 3\sigma_0.
\end{array}\right.$$

Suppose we want to denoise an image corrupted with noise $\sigma$, but instead of using the DCT denoiser with parameters corresponding to $\sigma_0$, we can only use the DCT denoiser for $\sigma_0$.

**Question 3.** Show that 
$$\text{DCT}_{\sigma_0/\alpha}(P_x u) = \text{DCT}_{\sigma_0}(\alpha u)\,/\,\alpha$$
and that 
$$\text{DCT}_{\sigma_0}(P_x u) + \beta \mathbf{1} = DCT_{\sigma_0}(P_xu + \beta\mathbf 1),$$
where $\alpha,\beta\in \mathbb R$ and $\mathbf 1 = (1,1,...,1)^T\in \mathbb R^d$ refers to a constant patch of ones. <!--Use this property to denoise an image with noise $\sigma$ with $\text{DCT}_{\sigma_0}$.-->

ANSWER TO QUESTION 3

The following experiment uses this property to denoise an image with noise $\sigma$ with $\text{DCT}_{\sigma_0}$. 


In [ ]:
from models import DCTlike
from denoising_helpers import *
from vistools import unzip

# load an image (change the number to test other images)
image = io.imread('datasets/BSD68/test002.png', dtype='float32')

# set a noise level
sigma0 = 30

im_clean = image.astype('float32')
noise = np.random.normal(0, 1., im_clean.shape)

# load pre-trained DnCNN
dct_sigma0 = DCTlike(8, sigma0, True)

sigmas = np.linspace(10,75,14) #sigmas = np.linspace(5,80,76)
#print(sigmas)

PSNR_sigma0 = np.zeros(len(sigmas))
PSNR_sigma  = np.zeros(len(sigmas))
PSNR_scaled = np.zeros(len(sigmas))

outputs = list()

# apply them to the image
for i, sigma in enumerate(sigmas):
    print('%d '%i, end='')
    
    # add noise
    im_noisy = im_clean + sigma * noise

    # load correct model
    dct_sigma = DCTlike(8, sigma, True)
    
    # denoise image with both models
    out_sigma0 = test_denoiser(dct_sigma0, im_noisy, None, has_noise=True)[0]
    out_sigma  = test_denoiser(dct_sigma , im_noisy, None, has_noise=True)[0]
    
    # scaling to match model noise level
    a = sigma0/sigma
    b = np.mean(im_noisy)*(1-a)
    out_scaled = test_denoiser(dct_sigma0, a*im_noisy + b, None, has_noise=True)[0]
    out_scaled = 1/a*out_scaled - b/a
    
    PSNR_sigma0[i] = PSNR(out_sigma0, im_clean)
    PSNR_sigma [i] = PSNR(out_sigma , im_clean)
    PSNR_scaled[i] = PSNR(out_scaled, im_clean)

    outputs.append( (out_sigma0,'noise sigma = %f - model sigma = %f - PSNR = %f' % (sigma, sigma0, PSNR_sigma0[i])) )
    outputs.append( (out_sigma ,'noise sigma = %f - model sigma = %f - PSNR = %f' % (sigma, sigma , PSNR_sigma [i])) )
    outputs.append( (out_scaled,'noise sigma = %f - scaled to sigma %f - PSNR = %f' % (sigma, sigma , PSNR_scaled[i])) )
    
    
# show as a gallery
if len(outputs) < 80:
    vistools.display_gallery( unzip(outputs,0), unzip(outputs,1) )

plt.plot(sigmas, PSNR_sigma0, '.-', label='model with sigma0')
plt.plot(sigmas, PSNR_sigma , '.-', label='model with correct sigma')
plt.plot(sigmas, PSNR_scaled, '.-', label='model with sigma0, scaled input')
plt.legend(); plt.xlabel('sigma'); plt.ylabel('PSNR (dB)');

# FFDNet

We will now study FFDNet proposed as an improvement to DnCNN by the same group:

*K. Zhang, W. Zuo, and L. Zhang, “FFDNet: Toward a Fast and Flexible Solution for {CNN} based Image Denoising,” CoRR, vol. abs/1710.0, 2017.*

<img width=700 src="https://raw.githubusercontent.com/gfacciol/MVAdenoising2018/master/models/ffdnet.png"/>

FFDNet consist of a small version of DnCNN, but it has two main differences. The first is the introduction of a noise variance map as an additional input. This map is an image $\sigma(x)$ that specifies the variance of the noise at location $x$. This allows treating noise with space-varying variance. The second difference is that an invertible downscaling transform is applied to the input image. This downscaling is in fact a re-ordering of the pixels of a $W\times H$ image as a 4-channel $W/2\times H/2$ images.

The implementation of FFDNet can be found in the `models/FFDNet.py` file. The model constructor can be called as follows:

```python
class FFDNet(nn.Module):
    def __init__(self, sigma=30/255, in_channels=1, out_channels=1, num_layers=15, 
                 features=64, kernel_size=3):
        '''
        Model for a FFDNet network build using CONV_BN_RELU units
        this network can handle any level of noise when sigma_map is provided.

        Args:
            - sigma       : default noise level when no sigma_map is given
            - in_channels : number of input image channels
            - out_channels: number of output image channels
            - num_layers  : number of layers of the inner DnCNN network
            - features    : number of features of hidden layers in the inner DnCNN net
            - kernel_size : kernel size for all layers in the inner DnCNN net
        '''
```

We provide you with a function to load pretrained weights
```python
    def FFDNet_pretrained_grayscale(sigma=30, savefile=None, verbose=False):
        '''
        Loads the pretrained weights of FFDNet for grayscale images from 
        https://github.com/cszn/FFDNet.git

        Args:
            - sigma   : is the default noise level for the network when sigma_map is not specified
            - savefile: is the .pt file to save the model weights 
            - verbose : verbose output
            
        Returns:
            - FFDNet(1,1) model with 15 layers with the pretrained weights
        '''
```

In [ ]:
# denoise image with both models

from models import FFDNet, FFDNet_pretrained_grayscale
from models import DnCNN, DnCNN_pretrained
from skimage import io
from denoising_helpers import test_denoiser

sigma=5
im_clean = io.imread('datasets/BSD68/test020.png', dtype='float32') 
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)

ffdnet = FFDNet_pretrained_grayscale(sigma)
dncnn  = DnCNN_pretrained(sigma)

outputs = list()

outputs.append( (im_noisy, 'noisy input with sigma = %f' % (sigma)) )
outputs.append( (im_clean, 'clean image') )

out = test_denoiser(dncnn, im_noisy, None, has_noise=True)[0]
outputs.append( (out, 'dncnn output - PSNR = %f' % PSNR(out, im_clean)) )

out = test_denoiser(ffdnet, im_noisy, None, has_noise=True)[0]
outputs.append( (out, 'ffdnet output - PSNR = %f' % PSNR(out, im_clean)) )

vistools.display_gallery(unzip(outputs,0), unzip(outputs,1) )

We will repeat the experiment that we did before and compare the performance of DnCNN and FFDNet when varying the noise level. For DnCNN we compute the result with a network trained with the correct noise level $\sigma$ and with a fixed noise level $\sigma_0$. Analogously, for FFDNet we compute the result with a noise map having the correct value 
$\sigma$ and one having a fixed noise level $\sigma_0$.





<br/>
<br/>

**Attention:** For performance reasons, the variables `PSNR_dncnn_sigma0` and `PSNR_dncnn_sigma` of this block are re-used from the block: *"Stress tests on DnCNN: robustness to changes in $\sigma$"*, that was executed before.

In [ ]:
from models import DnCNN, DnCNN_pretrained
from models import FFDNet, FFDNet_pretrained_grayscale
from denoising_helpers import *
from skimage import io
from vistools import unzip

# load an image (change the number to test other images)
image = io.imread('datasets/BSD68/test002.png', dtype='float32')

# set a noise level
sigma0 = 30

im_clean = image.astype('float32')
noise = np.random.normal(0, 1., im_clean.shape)

# load pre-trained DnCNN
dncnn_sigma0  =  DnCNN_pretrained(sigma0)
ffdnet = FFDNet_pretrained_grayscale(sigma0)

sigmas = np.linspace(10,75,14)  #sigmas = np.linspace(5,80,76)
#print(sigmas)

##############
### THE PSNR_dncnn_sigma0 and PSNR_dncnn_sigma VARIABLES ARE COMPUTED IN SECTION: 
###    Stress tests on DnCNN: robustness to changes in $\sigma$ 
##############

#PSNR_dncnn_sigma0  = np.zeros(len(sigmas))
#PSNR_dncnn_sigma   = np.zeros(len(sigmas))
PSNR_ffdnet_sigma0 = np.zeros(len(sigmas))
PSNR_ffdnet_sigma  = np.zeros(len(sigmas))

outputs = list()

# apply them to the image
for i, sigma in enumerate(sigmas):
    print('%d '%i, end='')
    
    # add noise
    im_noisy = im_clean + sigma * noise

    # denoise image with both DnCNN models
    dncnn_sigma = DnCNN_pretrained(sigma)
    #out_dncnn_sigma0 = test_denoiser(dncnn_sigma0, im_noisy, None, has_noise=True)[0]
    #out_dncnn_sigma  = test_denoiser(dncnn_sigma , im_noisy, None, has_noise=True)[0] 

    # denoise image with both FFDnet models
    out_ffdnet_sigma0 = test_denoiser(ffdnet, im_noisy, sigma0, has_noise=True, sigma_param=True)[0]
    out_ffdnet_sigma  = test_denoiser(ffdnet, im_noisy, sigma , has_noise=True, sigma_param=True)[0]
    
    #PSNR_dncnn_sigma0[i]  = PSNR(out_dncnn_sigma0 , im_clean)
    #PSNR_dncnn_sigma[i]   = PSNR(out_dncnn_sigma  , im_clean)
    PSNR_ffdnet_sigma0[i] = PSNR(out_ffdnet_sigma0, im_clean)
    PSNR_ffdnet_sigma[i]  = PSNR(out_ffdnet_sigma , im_clean)

    #outputs.append((out_dncnn_sigma0 , 'noise sigma = %4.1f - dncnn  sigma = %4.1f - PSNR = %5.2f' % (sigma, sigma0, PSNR_dncnn_sigma0 [i])))
    outputs.append((out_ffdnet_sigma0, 'noise sigma = %4.1f - ffdnet sigma = %4.1f - PSNR = %5.2f' % (sigma, sigma0, PSNR_ffdnet_sigma0[i])))
    #outputs.append((out_dncnn_sigma  , 'noise sigma = %4.1f - dncnn  sigma = %4.1f - PSNR = %5.2f' % (sigma, sigma , PSNR_dncnn_sigma  [i])))
    outputs.append((out_ffdnet_sigma , 'noise sigma = %4.1f - ffdnet sigma = %4.1f - PSNR = %5.2f' % (sigma, sigma , PSNR_ffdnet_sigma [i])))
    
    
# show as a gallery
if len(outputs) < 80:
    vistools.display_gallery(unzip(outputs,0), unzip(outputs,1) )

plt.figure(figsize=(18, 16))
plt.plot(sigmas, PSNR_dncnn_sigma0 , '.-', label='dncnn with sigma0')
plt.plot(sigmas, PSNR_dncnn_sigma  , '.-', label='dncnn with correct sigma')
plt.plot(sigmas, PSNR_ffdnet_sigma0, '.-', label='ffdnet with sigma0')
plt.plot(sigmas, PSNR_ffdnet_sigma , '.-', label='ffdnet with correct sigma')

plt.plot(sigmas, PSNR_dncnn_scaled  , '.-', label='dncnn with scaled sigma')
plt.legend(); plt.xlabel('sigma'); plt.ylabel('PSNR (dB)');

Note that the PSNR for FFDNet is only slightly higher than the one of DnCNN with the correct sigma, or the scaled DnCNN. The advantage of FFDNet is its efficiency.

# The choice of the loss

Our following experiments deal with the choice of the loss. There are several losses to choose from, and this choice has an impact on the results. [This paper](https://arxiv.org/abs/1511.08861) presents an empirical study of different losses for the application of denoising. 

We will investigate some properties of the $L_1$ and the squared $L_2$ losses. In addition to the Gaussian noise model that we have used so far, we will introduce a different type of noise that help to highlight some of the differences between these loss functions.

### Uniform salt and pepper noise

The salt and pepper noise contaminates a random set of pixels of an image. A corrupted pixel can take the value 0 or 255, chosen randomly (hence the name salt and pepper). 

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f4/Noise_salt_and_pepper.png"/>

In our uniform salt and pepper noise a pixel is corrupted with probability $p$. The corrupted pixels are replaced with a value drawn from a uniform distribution in $[0,255]$. If we denote by $v(x)$ the corrupted pixel and $u(x)$ the clean pixel, the PDF of $v(x)$ given $u(x)$ is:

$$P(v(x)\,|\,u(x)) = (1-p)\delta_{u(x)} + p.$$

In practice with `numpy` we can corrupt an image with this noise with the following code:
```python
# binary mask of corrupted pixels (0 means corrupted)
mask = np.random.binomial(1, 1-p, im_clean.shape)

# image of uniform noise in [0, 255]
noise = np.random.uniform(0., 255., im_clean.shape)

# replace corrupted pixels with uniform noise
im_noisy = im_clean * mask + noise * (1 - mask)
```

<br>

We have implemented a data loader for training which currupts images with the uniform salt-and-pepper noise. The following code demonstrates its usage.

In [ ]:
from denoising_dataloaders import train_val_denoising_dataloaders
import vistools

# noise level
p = 0.4

# create the data loader - note 'noise_type' argument
train_loader, val_loader = \
    train_val_denoising_dataloaders('./datasets/Train400/',
                                     noise_sigma=p,
                                     crop_size=40,
                                     train_batch_size=16,
                                     noise_type='uniform-sp')

# visualize first mini-batch
X, Y = list(train_loader)[0]

# this helper function displays the patches in the mini-batch
print('\n\nA minibatch of noisy patches:')
vistools.display_patches(X)

print('\n\nAnd the corresponding noiseless patches:')
vistools.display_patches(Y)

**Question 4.** Compute the cummulative distribution function (CDF) of $v(x)\,|\,u(x)$ and mean and median. The median of a random variable $X$ is defined as the value $m$ such that 

$$P(X \geq m) \geq 1/2\quad\text{and}\quad P(X\leq m)\leq 1/2.$$

ANSWER TO QUESTION 4

### Gaussian noise denoisers trained with $L_1$ and $L_2$ losses

We have trained our tiny DnCNN network with the MSE (or squared L2 loss) and the L1 loss. The following block does the training. We will skip this block, load the pretrained networks, and compare the results on a test image.

In [ ]:
from models import DnCNN
from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel

sigma=30

# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False

# choose loss
loss_name = 'l1'
#loss_name = 'l2'

if training:
    # data
    trainloader, validationloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=40, 
                                               train_batch_size=128)
    # network model
    dncnn = DnCNN(in_channels=1, out_channels=1, 
                  num_layers=7, features=13, kernel_size=3, residual=True)
    
    # run the training loop using chosen loss
    loss_fn = nn.L1Loss() if loss_name == 'l1' else nn.MSELoss()
    dncnn, losst, lossv = trainmodel(dncnn, loss_fn, trainloader, validationloader, 
                                     num_epochs=2000, save_every=500, loss_every=100,  
                                     learning_rate=0.01, weight_decay=0.00001,
                                     filename='pre-trained-tp2/tiny_DnCNN_%s_' % (loss_name))

In [ ]:
# denoise image with both models
from models import DnCNN
from skimage import io
from denoising_helpers import test_denoiser, PSNR
from vistools import unzip

sigma=30
im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)

outputs = list()

outputs.append((im_noisy, 'noisy input with sigma = %f' % (sigma)))
outputs.append((im_clean, 'clean image'))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l2_2000.pt', map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'dncnn MSE loss - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l1_2000.pt', map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'dncnn l1 loss - PSNR = %f' % PSNR(out, im_clean)))

# show as a gallery
vistools.display_gallery(unzip(outputs,0), unzip(outputs,1))

**Question 5.** Give a brief qualitative comparison of the two results.

ANSWER TO QUESTION 5

### Uniform salt-and-pepper noise denoisers trained with $L_1$ and $L_2$ losses

As before we will compare networks pretrained with this type of noise using $L_1$ and $L_2$ losses.

In [ ]:
# denoise image with both models
from models import DnCNN
from skimage import io
from denoising_helpers import test_denoiser, PSNR
from vistools import unzip

p=0.4
im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 
mask = np.random.binomial(1, 1-p, im_clean.shape)
noise = np.random.uniform(0, 255., im_clean.shape)
im_noisy = im_clean * mask + noise * (1 - mask)

outputs = list()

outputs.append((im_noisy, 'noisy input with sigma = %f' % (sigma)))
outputs.append((im_clean, 'clean image'))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l1_usp_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'dncnn usp l1 loss - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l2_usp_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'dncnn usp l2 loss - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l2_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'dncnn gaussian MSE loss - PSNR = %f' % PSNR(out, im_clean)))

vistools.display_gallery(unzip(outputs,0), unzip(outputs,1))

It seems that both losses behave roughly the same for both types of considered noise. We will see a different training setup, in which the choice of the loss is crucial.

# Noise to noise training

We will now follow the training approached proposed in

*Noise2Noise: Learning Image Restoration without Clean Data*<br/>
J. Lehtinen, J. Munkberg, J. Hasselgren, S. Laine, T. Karras, M. Aittala, T. Aila. In ICML 2018.

The idea is here is to compute the loss between two noise realizations of the same clean image. To train
with this approach, we pass the flag `noise2noise=True` to the data loader. The data loader will then return pairs of noisy images.

In [ ]:
from denoising_dataloaders import train_val_denoising_dataloaders
import vistools

# noise level
p = 40

# create the data loader - note 'noise_type' argument
train_loader, val_loader = \
    train_val_denoising_dataloaders('./datasets/Train400/',
                                     noise_sigma=p,
                                     crop_size=40,
                                     train_batch_size=16,
                                     noise_type='gaussian',
                                     noise2noise=True)

# visualize first mini-batch
X, Y = list(train_loader)[0]

# this helper function displays the patches in the mini-batch
print('\n\nA minibatch of noisy patches:')
vistools.display_patches(X)

print('\n\nAnd the corresponding patches with a different realization of the noise:')
vistools.display_patches(Y)

In [ ]:
from models import DnCNN
from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel

p=30

loss_name = 'l1'
#loss_name = 'l2'

# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False

if training: 
    # data
    train_loader, val_loader = train_val_denoising_dataloaders(
                                              './datasets/Train400/',
                                               noise_sigma=p, crop_size=40, 
                                               train_batch_size=128,
                                               noise_type='gaussian',
                                               noise2noise=True)

    # network model
    dncnn = DnCNN(in_channels=1, out_channels=1, 
                  num_layers=7, features=13, kernel_size=3, residual=True)

    # run the training loop
    loss = nn.L1Loss() if loss_name == 'l1' else nn.MSELoss()
    dncnn, losst, lossv = trainmodel(dncnn, loss, train_loader, val_loader, 
                                    num_epochs=2000, save_every=500, loss_every=100,  
                                    learning_rate=0.01, weight_decay=0.00001,
                                    filename='pre-trained-tp2/tiny_DnCNN_n2n_%s_' % loss_name)

    # plot loss
    plt.semilogy(lossv, label='val')
    plt.semilogy(losst, label='train')
    plt.legend()

### Gaussian noise denoisers trained with $L_1$ and $L_2$ losses and noisy targets

We have trained our tiny DnCNN network with the MSE (or squared L2 loss) and the L1 loss following the noise2noise approach. Let's compare the results.

In [ ]:
# denoise image with both models
from models import DnCNN
from skimage import io
from denoising_helpers import test_denoiser, PSNR
from vistools import unzip

sigma=30
im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)

outputs = list()

outputs.append((im_clean, 'clean image'))
outputs.append((im_noisy, 'noisy input with sigma = %f' % (sigma)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_n2n_l2_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'l2 loss, n2n training - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_n2n_l1_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'l1 loss, n2n training - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l1_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'l1 loss - PSNR = %f' % PSNR(out, im_clean)))

vistools.display_gallery(unzip(outputs,0), unzip(outputs,1))

### Uniform S&P noise denoisers trained with $L_1$ and $L_2$ losses and noisy targets

We have trained our tiny DnCNN network with the MSE (or squared L2 loss) and the L1 loss following the noise2noise approach. Let's compare the results.

In [ ]:
# denoise image with both models
from models import FFDNet, FFDNet_pretrained_grayscale
from models import DnCNN, DnCNN_pretrained
from skimage import io
from denoising_helpers import test_denoiser, PSNR

p=0.4
im_clean = io.imread('datasets/BSD68/test006.png', dtype='float32') 
mask = np.random.binomial(1, 1-p, im_clean.shape)
noise = 255. * np.random.uniform(0, 1, im_clean.shape)
im_noisy = im_clean * mask + noise * (1 - mask)


outputs = list()

outputs.append((im_clean, 'clean image'))
outputs.append((im_noisy, 'noisy input with sigma = %f' % (sigma)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_n2n_l2_usp_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'l2 loss, n2n training - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_n2n_l1_usp_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'l1 loss, n2n training - PSNR = %f' % PSNR(out, im_clean)))

net = torch.load('pre-trained-tp2/tiny_DnCNN_l1_usp_2000.pt' , map_location=loadmap)[0]
out = test_denoiser(net, im_noisy, None, has_noise=True)[0]
outputs.append((out, 'l1 loss - PSNR = %f' % PSNR(out, im_clean)))


vistools.display_gallery(unzip(outputs,0), unzip(outputs,1))

**Question 6.** When training using noisy targets with Gaussian noise, is there any difference between the results obtained with both losses? What about the uniform S&P noise? Give an explanation for this behaviour.

ANSWER TO QUESTION 6

---------------------------
[//]: # (© 2018 Gabriele Facciolo and Pablo Arias)
[//]: # (<div style="text-align:center; font-size:75%;"> Copyright © 2018 Gabriele Facciolo and Pablo Arias. All rights reserved.</div> )